In [1]:
1*2

2

In [2]:
import sys
sys.path.insert(1, '../')

import os
import GPUtil
import importlib

def sgpu():
    GPUtil.showUtilization()

def rl(module):
    importlib.reload(module)

In [3]:
sgpu()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  0% |  0% |
|  2 |  0% |  0% |
|  3 |  0% |  0% |
|  4 |  0% |  0% |
|  5 |  0% |  0% |
|  6 |  0% |  0% |
|  7 |  0% |  0% |


In [4]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import torch
import gpytorch

from scipy.stats import pearsonr
from scipy.special import binom as binom
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split

In [5]:
import EpiK.models as models

In [6]:
output_device = 0
n_devices = torch.cuda.device_count()
models.set_params(output_device, n_devices)
print("number of GPUs = {}; output device = {}".
      format(n_devices, torch.cuda.current_device()))

number of GPUs = 8; output device = 0


In [7]:
from EpiK.functions import get_data, get_envs, set_data_path
set_data_path("../matsui_data/")

### Data

In [8]:
env_list = get_envs()
env = env_list[5]

In [9]:
geno_t, pheno = get_data(env)

../EpiK/functions.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  geno_t = torch.tensor(geno_t, dtype=torch.float)


In [10]:
inds_sub = np.where(np.array(pheno.pheno < -0.6) == False)[0]

In [11]:
import EpiK.functions
rl(EpiK.functions)
from EpiK.functions import train_model_cv

EpiK.functions.output_device = output_device
EpiK.functions.n_devices = n_devices

In [12]:
from EpiK.kernels import DiKernel
ker = DiKernel()
ker.raw_lda = torch.nn.Parameter(torch.tensor(-8.))
ker.raw_eta = torch.nn.Parameter(torch.tensor(-12.))

In [13]:
# training sizes
props = [.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .99]

# check_point sizes
partitions = [2, 2, 2, 2, 4, 4, 4, 4, 16, 32, 64, 80]

In [44]:
i=10

# define training data
np.random.seed(100)
train_size = np.round(props[i]*len(inds_sub)).astype('int')
sub = np.random.choice(inds_sub, train_size)
sub_t = np.random.choice(list(set(inds_sub).difference(sub)), 5000)
train_x = geno_t[sub]
train_y = torch.tensor(np.array(pheno.pheno[sub]), dtype=torch.float32)
test_x = geno_t[sub_t]
test_y = torch.tensor(np.array(pheno.pheno[sub_t]), dtype=torch.float32)
train_x, train_y = train_x.contiguous(), train_y.contiguous()
test_x, test_y = test_x.contiguous(), test_y.contiguous()
train_x, train_y = train_x.to(output_device), train_y.to(output_device)
test_x, test_y = test_x.to(output_device), test_y.to(output_device)

# # train kernel
# ker, likelihood = train_model_cv(ker, train_x, train_y, 50, .1)

# print("raw_lda = %f"%round(ker.state_dict()['raw_lda'].item(),3), "and", "raw_eta = %f"%round(ker.state_dict()['raw_eta'].item(),3))


sgpu()

likelihood = gpytorch.likelihoods.GaussianLikelihood().to(output_device)
model = models.ExactGPModel(train_x, train_y, likelihood, ker).to(output_device)


test_x = test_x.cuda()
model.eval()
likelihood.eval()

with gpytorch.beta_features.checkpoint_kernel(0):
    f_preds = model(test_x)

f_mean = f_preds.mean.cpu().detach().numpy()
y_test = test_y.detach().cpu().numpy()
r2_score = r2(y_test, f_mean)
print('training size = %i'%train_size,'; R2 = %f'%r2_score)
r2_score_list.append(r2_score)

del model, likelihood, f_preds
torch.cuda.empty_cache()

sgpu() 

| ID | GPU | MEM |
------------------
|  0 |  0% | 23% |
|  1 |  0% |  9% |
|  2 |  0% |  9% |
|  3 |  0% |  9% |
|  4 |  0% |  9% |
|  5 |  0% |  9% |
|  6 |  0% |  9% |
|  7 |  0% |  9% |


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/juannanzhou/.local/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "../EpiK/kernels.py", line 308, in __call__
    res = lazify(super(Kernel, self).__call__(x1_, x2_, last_dim_is_batch=last_dim_is_batch, **params))
  File "/home/juannanzhou/.local/lib/python3.8/site-packages/gpytorch/module.py", line 30, in __call__
    outputs = self.forward(*inputs, **kwargs)
  File "../EpiK/kernels.py", line 531, in forward
    S2 = self.covar_dist(geno1_h0, geno2_h0, **params) + self.covar_dist(geno1_h1, geno2_h1, **params)
RuntimeError: CUDA out of memory. Tried to allocate 13.32 GiB (GPU 0; 79.35 GiB total capacity; 76.87 GiB already allocated; 237.19 MiB free; 77.30 GiB reserved in total by PyTorch)


In [47]:
sgpu()

| ID | GPU  | MEM |
-------------------
|  0 |   0% | 29% |
|  1 |   0% |  9% |
|  2 |   0% |  9% |
|  3 |  24% |  9% |
|  4 |  79% |  9% |
|  5 | 100% |  9% |
|  6 | 100% |  9% |
|  7 | 100% |  9% |


In [46]:
def delete(list):
    for item in list:
        try: del item
        except: pass
delete([train_y, train_y, model, ker, likelihood, test_x, test_y])

import gc
gc.collect()
torch.cuda.empty_cache()

In [7]:
x=-10

In [10]:
import numpy as np
try: np.log(x)
except: 
    x +=1
    try: np.log(x)
    except: 
        x +=1
        np.log(x)

/scratch/local/44345443/ipykernel_161866/2455573701.py:2: RuntimeWarning: invalid value encountered in log
  try: np.log(x)


In [11]:
x

-10

In [ ]:
import numpy as np

while True:
    try: np.log(x)
    except: 
        x +=1
        np.log(x)

/scratch/local/44345443/ipykernel_160037/4139848123.py:4: RuntimeWarning: invalid value encountered in log
  try: np.log(x)


In [ ]:
loop = True
while loop:
    try:
        #Your code here
        loop = False
    except:
        #Your other code here

In [3]:
import numpy as np
loop = True
x = 0

In [15]:
while loop:
    try: 
        try: 1/np.round(x)
        except: loop = False
    except: x+=.1

In [7]:
x = 0

In [8]:
loop = True
while loop:
    try: 
        1/ (x//1)
        loop = False
    except: x = x + .1

In [9]:
x

1.0999999999999999